In [50]:
from collections import deque
import numpy as np
import heapq
from collections import namedtuple
State = namedtuple('State', ['state', 'g_cost', 'h_cost', 'path'])

## Generation of the random matrix of N dimension

In [51]:
#just a function to check if the randomly generated matrix is solvable or not
def is_solvable(puzzle, n):
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(1 for i in range(len(flattened)) for j in range(i+1, len(flattened)) if flattened[i]>flattened[j])
    if n%2 != 0:
        return inversions % 2 == 0
    else:
        blank_row = n -np.where(puzzle == 0)[0][0]
        return (blank_row % 2 == 0) != (inversions %2 == 0)                                  

In [52]:
def matrix_generator(n):
    while True:
        puzzle = np.random.permutation(n**2).reshape(n,n)
        if is_solvable(puzzle, n):
            return puzzle

## Implementaion of A*

In [53]:
# Manhattan Distance Heuristic
def manhattan_distance(state, goal_flat):
    """Calculate Manhattan distance between a current state and the goal."""
    n = state.shape[0]
    distance = 0
    for i in range(n):
        for j in range(n):
            value = state[i, j]
            if value != 0:
                goal_index = goal_flat.index(value)
                goal_x, goal_y = divmod(goal_index, n)
                distance += abs(i - goal_x) + abs(j - goal_y)
    return distance

In [54]:
#generation of neighbouring states by moving the blank tilt

def get_neighbors(state):
    n = state.shape[0]
    neighbors = []
    x, y = np.argwhere(state == 0)[0]
    moves = [(-1, 0), (1,0), (0, -1), (0,1)] #top , bottom, Left, right
    for dx, dy in moves:
        nx, ny = x+dx, y+dy
        if 0<=nx <n and 0 <= ny <n:
            new_state = state.copy()
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x,y]
            neighbors.append(new_state)
    return neighbors

In [55]:
def state_to_tuple(state):
    return tuple(state.flatten())

In [56]:
# Bidirectional A* Search function
def bidirectional_a_star(start, goal):
    n = start.shape[0]
    goal_flat = [tile for row in goal for tile in row]  # Flatten the goal for heuristic

    # Priority queues for both searches (start and goal)
    frontier_start = []
    frontier_goal = []
    
    # Visited dictionaries for both directions
    visited_start = {}
    visited_goal = {}

    # Add the initial states to the frontier
    heapq.heappush(frontier_start, (manhattan_distance(start, goal_flat), 0, state_to_tuple(start), [start]))
    heapq.heappush(frontier_goal, (manhattan_distance(goal, goal_flat), 0, state_to_tuple(goal), [goal]))
    
    visited_start[state_to_tuple(start)] = (0, [start])  # (cost, path)
    visited_goal[state_to_tuple(goal)] = (0, [goal])  # (cost, path)

    while frontier_start and frontier_goal:
        # Expand the frontiers from both directions
        f_start, g_start, current_start_tuple, path_start = heapq.heappop(frontier_start)
        f_goal, g_goal, current_goal_tuple, path_goal = heapq.heappop(frontier_goal)

        # Reconstruct the numpy arrays from the tuples
        current_start = np.array(current_start_tuple).reshape(n, n)
        current_goal = np.array(current_goal_tuple).reshape(n, n)
        
        # Check if the two searches meet (intersection)
        if current_start_tuple in visited_goal:
            path_goal_back = visited_goal[current_start_tuple][1]
            return path_start + path_goal_back[::-1], g_start + g_goal

        if current_goal_tuple in visited_start:
            path_start_back = visited_start[current_goal_tuple][1]
            return path_goal + path_start_back[::-1], g_start + g_goal

        # Expand from the start search
        for neighbor in get_neighbors(current_start):
            neighbor_tuple = state_to_tuple(neighbor)
            if neighbor_tuple not in visited_start:
                heapq.heappush(frontier_start, (
                    g_start + 1 + manhattan_distance(neighbor, goal_flat),
                    g_start + 1,
                    neighbor_tuple,
                    path_start + [neighbor]
                ))
                visited_start[neighbor_tuple] = (g_start + 1, path_start + [neighbor])

        # Expand from the goal search
        for neighbor in get_neighbors(current_goal):
            neighbor_tuple = state_to_tuple(neighbor)
            if neighbor_tuple not in visited_goal:
                heapq.heappush(frontier_goal, (
                    g_goal + 1 + manhattan_distance(neighbor, goal_flat),
                    g_goal + 1,
                    neighbor_tuple,
                    path_goal + [neighbor]
                ))
                visited_goal[neighbor_tuple] = (g_goal + 1, path_goal + [neighbor])

    return None, 0

In [57]:
PUZZLE_DIM = 4
start = matrix_generator(PUZZLE_DIM)
goal = np.arange(1, PUZZLE_DIM**2).tolist() + [0]
goal = np.array(goal).reshape(PUZZLE_DIM, PUZZLE_DIM)

print("Initial puzzle state:\n", start)
print("Goal puzzle state:\n", goal)

# Run Bidirectional A* Search
solution_path, cost = bidirectional_a_star(start, goal)

if solution_path:
    print(f"Solution found with {len(solution_path)} moves!")
    for step, state in enumerate(solution_path):
        print(f"Step {step + 1}:\n{state}")
else:
    print("No solution found.")
print(f"Quality (Number of Moves): {len(solution_path)}") #length of the solution path in case of A* it is equivalent to the length of the solution path.
print(f"Cost (Nodes Evaluated): {cost}")
print(f"Efficiency (Quality / Cost): {len(solution_path) / cost:.4f}")



Initial puzzle state:
 [[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 10  8  0]
 [14 13  4 12]]
Goal puzzle state:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Solution found with 49 moves!
Step 1:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 10  8  0]
 [14 13  4 12]]
Step 2:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 10  0  8]
 [14 13  4 12]]
Step 3:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5  0 10  8]
 [14 13  4 12]]
Step 4:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 13 10  8]
 [14  0  4 12]]
Step 5:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 13 10  8]
 [14  4  0 12]]
Step 6:
[[ 9  3  6  2]
 [ 1 11 15  7]
 [ 5 13  0  8]
 [14  4 10 12]]
Step 7:
[[ 9  3  6  2]
 [ 1 11  0  7]
 [ 5 13 15  8]
 [14  4 10 12]]
Step 8:
[[ 9  3  0  2]
 [ 1 11  6  7]
 [ 5 13 15  8]
 [14  4 10 12]]
Step 9:
[[ 9  0  3  2]
 [ 1 11  6  7]
 [ 5 13 15  8]
 [14  4 10 12]]
Step 10:
[[ 0  9  3  2]
 [ 1 11  6  7]
 [ 5 13 15  8]
 [14  4 10 12]]
Step 11:
[[ 1  9  3  2]
 [ 0 11  6  7]
 [ 5 13 15  8]
 [14  4 10 12]]
Step 12:
[[ 1  9  3  2]
 [ 5 11  6  7]
 [ 0 